## DOW30 Galaxy

Use this utlity to update the returns and std_dev fields within investment-options.csv

Globals

In [1]:
# Set refresh_timeseries=True to download timeseries.  Otherwise /symbol-cache is used.
refresh_timeseries = True
throttle_limit=100
wait_time=30

In [2]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import brownbear as bb

# Format price data.
pd.options.display.float_format = '{:0.2f}'.format

In [4]:
# Read in dow30.csv
dow30 = pd.read_csv('dow30.csv')

# Remove the exchange from the beginning of the symbol.
def _symbol(row):
    return row['Symbol'].split(':')[-1].strip()

dow30['Symbol'] = dow30.apply(_symbol, axis=1)
dow30.drop(columns=['Exchange', 'Date added', 'Notes', 'Index weighting'], inplace=True)
dow30.rename(columns={'Symbol': 'Symbol',
                      'Company':'Description',
                      'Industry':'Asset Class'}, inplace=True)

dow30.set_index("Symbol", inplace=True)
dow30

,Description,Asset Class
Symbol,,
MMM,3M,Conglomerate
AXP,American Express,Financial services
AMGN,Amgen,Biopharmaceutical
AMZN,Amazon,Retailing
AAPL,Apple,Information technology
BA,Boeing,Aerospace and defense
CAT,Caterpillar,Construction and mining
CVX,Chevron,Petroleum industry
CSCO,Cisco,Information technology


In [5]:
# Read in gics-2-asset-class.csv
gics2asset_class = pd.read_csv('gics-2-asset-class.csv', skip_blank_lines=True, comment='#')
gics2asset_class.set_index("GICS", inplace=True)
gics2asset_class = gics2asset_class['Asset Class'].to_dict()
gics2asset_class

{'Conglomerate': 'US Stocks:Industrials',
 'Financial services': 'US Stocks:Financials',
 'Pharmaceutical industry': 'US Stocks:Healthcare',
 'Biopharmaceutical': 'US Stocks:Healthcare',
 'Information technology': 'US Stocks:Technology',
 'Aerospace and defense': 'US Stocks:Industrials',
 'Construction and mining': 'US Stocks:Industrials',
 'Petroleum industry': 'US Stocks:Energy',
 'Food industry': 'US Stocks:Consumer Staples',
 'Drink industry': 'US Stocks:Consumer Staples',
 'Chemical industry': 'US Stocks:Materials',
 'Retailing': 'US Stocks:Consumer Discretionary',
 'Apparel': 'US Stocks:Consumer Discretionary',
 'Fast-moving consumer goods': 'US Stocks:Consumer Staples',
 'Managed health care': 'US Stocks:Healthcare',
 'Telecommunication': 'US Stocks:Communication Services',
 'Broadcasting and entertainment': 'US Stocks:Communication Services',
 'Home Improvement': 'US Stocks:Consumer Discretionary',
 'Semiconductor industry': 'US Stocks:Technology',
 'Clothing industry': 'US Sto

In [6]:
# Map dow30 GICS sectors to brownbear defined asset classes.
def _asset_class(row):
    return gics2asset_class[row['Asset Class']]

dow30['Asset Class'] = dow30.apply(_asset_class, axis=1)

# Yahoo finance uses '-' where '.' is used in symbol names.
dow30.index = dow30.index.str.replace('.', '-', regex=False)
dow30

,Description,Asset Class
Symbol,,
MMM,3M,US Stocks:Industrials
AXP,American Express,US Stocks:Financials
AMGN,Amgen,US Stocks:Healthcare
AMZN,Amazon,US Stocks:Consumer Discretionary
AAPL,Apple,US Stocks:Technology
BA,Boeing,US Stocks:Industrials
CAT,Caterpillar,US Stocks:Industrials
CVX,Chevron,US Stocks:Energy
CSCO,Cisco,US Stocks:Technology


In [7]:
# Make symbols list.
symbols = list(dow30.index)
#symbols

In [8]:
# Get the timeseries for the symbols and compile into a single csv.
bb.fetch_timeseries(symbols, refresh=refresh_timeseries, throttle_limit=throttle_limit, wait_time=wait_time)
bb.compile_timeseries(symbols)

MMM AXP AMGN AMZN AAPL BA CAT CVX CSCO KO DIS GS HD HON IBM JNJ JPM MCD MRK MSFT NKE NVDA PG CRM SHW TRV UNH VZ V WMT 


In [9]:
# Read symbols timeseries into a dataframe.
df = pd.read_csv('symbols-timeseries.csv', skip_blank_lines=True, comment='#')
df.set_index("Date", inplace=True)
df

,MMM,AXP,AMGN,AMZN,AAPL,BA,CAT,CVX,CSCO,KO,...,NKE,NVDA,PG,CRM,SHW,TRV,UNH,VZ,V,WMT
Date,,,,,,,,,,,,,,,,,,,,,
2019-01-02,125.57,87.53,158.59,76.96,37.67,314.65,109.19,84.23,35.27,38.67,...,68.84,3.38,77.52,134.59,123.63,102.22,221.53,39.45,127.27,28.22
2019-01-03,120.84,85.83,156.17,75.01,33.92,302.10,104.99,82.62,33.98,38.43,...,67.62,3.17,76.97,129.47,119.90,100.38,215.49,39.59,122.69,28.07
2019-01-04,125.81,89.69,161.51,78.77,35.36,317.82,110.72,84.33,35.51,39.20,...,69.39,3.38,78.54,136.98,123.08,103.36,218.01,39.69,127.97,28.25
2019-01-07,125.52,90.18,163.69,81.48,35.28,318.82,110.79,85.43,35.75,38.69,...,70.38,3.56,78.23,141.21,124.81,103.03,218.43,39.94,130.28,28.58
2019-01-08,126.05,90.62,165.78,82.83,35.96,330.89,112.12,85.05,36.04,39.13,...,71.32,3.47,78.52,144.68,125.70,102.63,221.35,41.11,130.99,28.78
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-04-28,137.93,264.95,283.09,187.70,210.14,182.30,307.06,140.10,56.84,71.79,...,57.31,108.73,161.85,265.64,332.20,261.61,420.00,42.39,337.51,95.22
2025-04-29,138.38,267.02,288.56,187.39,211.21,182.00,307.40,139.30,57.34,72.35,...,57.54,109.02,162.26,267.76,348.13,264.05,409.23,42.95,341.52,96.04
2025-04-30,138.91,266.41,290.92,184.42,212.50,183.24,309.27,136.06,57.73,72.55,...,56.40,108.92,162.57,268.71,352.92,264.13,411.44,44.06,345.50,97.25


In [10]:
# Calculate Annual Returns.
annual_returns = bb.annualized_returns(df, timeperiod='daily', years=1)
annual_returns

MMM     50.57
AXP     19.63
AMGN     5.79
AMZN     8.56
AAPL    21.13
BA      10.50
CAT     -1.68
CVX    -10.40
CSCO    30.08
KO      19.46
DIS    -16.00
GS      35.59
HD      11.78
HON     13.46
IBM     52.71
JNJ     11.44
JPM     34.75
MCD     17.03
MRK    -33.72
MSFT    12.66
NKE    -35.31
NVDA    32.56
PG       0.76
CRM      2.88
SHW     21.04
TRV     28.28
UNH    -16.00
VZ      18.23
V       30.37
WMT     68.27
dtype: float64

In [11]:
# Calculate 1 month, 3 months, 1 year, 3 year, and 5 year annualized returns.
annual_returns_1mo = bb.annualized_returns(df, timeperiod='daily', years=1/12)
annual_returns_3mo = bb.annualized_returns(df, timeperiod='daily', years=3/12)
annual_returns_1yr = bb.annualized_returns(df, timeperiod='daily', years=1)
annual_returns_3yr = bb.annualized_returns(df, timeperiod='daily', years=3)
annual_returns_5yr = bb.annualized_returns(df, timeperiod='daily', years=5)

In [12]:
# Calculate 20 day annualized volatility.
daily_returns = df.pct_change()
years = bb.TRADING_DAYS_PER_MONTH / bb.TRADING_DAYS_PER_YEAR
vola = bb.annualized_standard_deviation(daily_returns, timeperiod='daily', years=years)

In [13]:
# Calculate 20 day annualized downside volatility.
ds_vola = bb.annualized_standard_deviation(daily_returns, timeperiod='daily', years=years, downside=True)

In [14]:
# Resample df on a monthly basis.
df.index = pd.to_datetime(df.index)
monthly = df.resample('ME').ffill()

In [15]:
# Calculate monthly returns.
monthly_returns = monthly.pct_change()

In [16]:
# Calculate 1 year, 3 year, and 5 year annualized standard deviation.
std_dev_1yr = bb.annualized_standard_deviation(monthly_returns, timeperiod='monthly', years=1)
std_dev_3yr = bb.annualized_standard_deviation(monthly_returns, timeperiod='monthly', years=3)
std_dev_5yr = bb.annualized_standard_deviation(monthly_returns, timeperiod='monthly', years=5)

In [17]:
# Read investment-options-header.csv
lines = []
with open('investment-options-in.csv', 'r') as f:
    lines = [line.strip() for line in f]

In [18]:
# For each symbol, write out the 1 Yr, 3 Yr, 5 Yr, and std dev.
out = lines.copy()

for i, (index, row) in enumerate(dow30.iterrows()):

    symbol = index
    description = row['Description']
    asset_class = row['Asset Class']

    ret_1mo = annual_returns_1mo[symbol]
    ret_3mo = annual_returns_3mo[symbol]
    ret_1yr = annual_returns_1yr[symbol]
    ret_3yr = annual_returns_3yr[symbol]
    ret_5yr = annual_returns_5yr[symbol]
    
    if np.isnan(ret_3yr): ret_3yr = ret_1yr
    if np.isnan(ret_5yr): ret_5yr = ret_3yr

    _vola = vola[symbol]*100
    _ds_vola = ds_vola[symbol]*100
    sd_1yr = std_dev_1yr[symbol]*100
    sd_3yr = std_dev_3yr[symbol]*100
    sd_5yr = std_dev_5yr[symbol]*100

    out.append((
        '"{}","{}","{}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}",'
        '"{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}"'
    ).format(
        symbol, description, asset_class, ret_1mo, ret_3mo, ret_1yr, ret_3yr,
        ret_5yr, _vola, _ds_vola, sd_1yr, sd_3yr, sd_5yr
    ))

In [19]:
# Write out asset-classes.csv
with open('investment-options.csv', 'w') as f:
    for line in out:
        f.write(line + '\n')